In [25]:
import pandas as pd

In [26]:
Label_df = pd.read_csv('egd_Labels.txt',names=['ID','label','extra'],sep='\t', dtype={'ID':str,'label':int,'extra':int})
df = pd.read_csv('Annotated_CombinedDirCsv.csv')

In [27]:
df.head()

,structured_path,filename,path,raw_name,name,label_exp,subject,train,NIFTI_path
0,data/egd/DICOM_STRUCTURED/MR_20020205/20020205...,1.2.826.0.1.3680043.10.104.2857427232523006061...,data/egd/data/MR_20020205_EGD-0924/scans/3-T1_...,3-T1_3D,T1w,MR_20020205_EGD-0924,EGD-0924,False,data/egd/NIFTI/MR_20020205/20020205/MR/1.2.826...
1,data/egd/DICOM_STRUCTURED/MR_20020205/20020205...,1.2.826.0.1.3680043.10.104.2857427232523006061...,data/egd/data/MR_20020205_EGD-0924/scans/8-T2_...,8-T2_Ax,T2w,MR_20020205_EGD-0924,EGD-0924,False,data/egd/NIFTI/MR_20020205/20020205/MR/1.2.826...
2,data/egd/DICOM_STRUCTURED/MR_61373/20170605/MR...,1.2.826.0.1.3680043.10.104.1044043140781656433...,data/egd/data/MR_20150326_EGD-1215/scans/3-3D_...,3-3D_Ax_T1_FSPGR_HR_blanco,T1w,MR_20150326_EGD-1215,EGD-1215,True,data/egd/NIFTI/MR_61373/20170605/MR/1.2.826.0....
3,data/egd/DICOM_STRUCTURED/MR_61373/20170605/MR...,1.2.826.0.1.3680043.10.104.1044043140781656433...,data/egd/data/MR_20150326_EGD-1215/scans/650-a...,650-ax_cube,other,MR_20150326_EGD-1215,EGD-1215,True,data/egd/NIFTI/MR_61373/20170605/MR/1.2.826.0....
4,data/egd/DICOM_STRUCTURED/MR_61373/20170605/MR...,1.2.826.0.1.3680043.10.104.1044043140781656433...,data/egd/data/MR_20150326_EGD-1215/scans/450-A...,450-ADC__10__6_mm__s__Nov_02_2018_20_57_56_CET,adc,MR_20150326_EGD-1215,EGD-1215,True,data/egd/NIFTI/MR_61373/20170605/MR/1.2.826.0....


In [28]:
df.describe()

,structured_path,filename,path,raw_name,name,label_exp,subject,train,NIFTI_path
count,2442,2442,2442,2442,2442,2442,2442,2442,2442
unique,2370,2361,2424,856,18,260,104,2,2370
top,data/egd/DICOM_STRUCTURED/MR_29988/20170615/MR...,1.2.826.0.1.3680043.10.104.1002580906157172357...,data/egd/data/MR_20100131_EGD-0381/scans/9-T1G...,4-DWI_DWI_Ax,T1c,MR_20150526_EGD-1220,EGD-0119,True,data/egd/NIFTI/MR_29988/20170615/MR/1.2.826.0....
freq,2,4,2,53,501,27,162,2120,2


In [29]:
def extractNiftiFilepathAndSlicenum(df):
    ID = df['ID']
    split = ID.rsplit('__s', 1)
    NiftiPath = split[0]
    NiftiPath = NiftiPath.replace('NIFTI_SLICES', 'NIFTI') + '.nii.gz'
    slicenum = int(split[1].split('.nii.gz')[0])
    return pd.Series({'NiftiPath': NiftiPath, 'slicenum': slicenum})

In [30]:
appendstring = '/trinity/home/r098375/DDS/'
def fixPath(ID):
    if not 'trinity' in ID:
        ID = appendstring+ID
    basepart = '/'.join((ID.split('/')[:9]))
    namepart = '__'.join((ID.split('/')[9:]))
    ID = basepart+'/'+namepart+'.nii.gz'
    return ID

In [31]:
df['NIFTI_path'] = df['NIFTI_path'].apply(fixPath)

In [32]:
df['NIFTI_path'][0]

'/trinity/home/r098375/DDS/data/egd/NIFTI/MR_20020205/20020205__MR__1.2.826.0.1.3680043.10.104.2857427232523006061017867594417320946__1.2.826.0.1.3680043.10.104.1456467217459860599267812336880387893.nii.gz'

In [33]:
df_NS = Label_df.apply(extractNiftiFilepathAndSlicenum, axis=1, result_type='expand')
Label_df = Label_df.merge(df_NS, left_index=True, right_index=True, how='inner')

In [34]:
Label_df['NiftiPath'][0]

'/trinity/home/r098375/DDS/data/egd/NIFTI/MR_20020205/20020205__MR__1.2.826.0.1.3680043.10.104.2857427232523006061017867594417320946__1.2.826.0.1.3680043.10.104.2282046845047365453865806177133221068.nii.gz'

In [35]:
df_new = pd.merge(df,Label_df,how='inner',right_on='NiftiPath',left_on='NIFTI_path')

In [36]:
df_new

,structured_path,filename,path,raw_name,name,label_exp,subject,train,NIFTI_path,ID,label,extra,NiftiPath,slicenum
0,data/egd/DICOM_STRUCTURED/MR_20020205/20020205...,1.2.826.0.1.3680043.10.104.2857427232523006061...,data/egd/data/MR_20020205_EGD-0924/scans/3-T1_...,3-T1_3D,T1w,MR_20020205_EGD-0924,EGD-0924,False,/trinity/home/r098375/DDS/data/egd/NIFTI/MR_20...,/trinity/home/r098375/DDS/data/egd/NIFTI_SLICE...,0,0,/trinity/home/r098375/DDS/data/egd/NIFTI/MR_20...,11
1,data/egd/DICOM_STRUCTURED/MR_20020205/20020205...,1.2.826.0.1.3680043.10.104.2857427232523006061...,data/egd/data/MR_20020205_EGD-0924/scans/3-T1_...,3-T1_3D,T1w,MR_20020205_EGD-0924,EGD-0924,False,/trinity/home/r098375/DDS/data/egd/NIFTI/MR_20...,/trinity/home/r098375/DDS/data/egd/NIFTI_SLICE...,0,0,/trinity/home/r098375/DDS/data/egd/NIFTI/MR_20...,8
2,data/egd/DICOM_STRUCTURED/MR_20020205/20020205...,1.2.826.0.1.3680043.10.104.2857427232523006061...,data/egd/data/MR_20020205_EGD-0924/scans/3-T1_...,3-T1_3D,T1w,MR_20020205_EGD-0924,EGD-0924,False,/trinity/home/r098375/DDS/data/egd/NIFTI/MR_20...,/trinity/home/r098375/DDS/data/egd/NIFTI_SLICE...,0,0,/trinity/home/r098375/DDS/data/egd/NIFTI/MR_20...,21
3,data/egd/DICOM_STRUCTURED/MR_20020205/20020205...,1.2.826.0.1.3680043.10.104.2857427232523006061...,data/egd/data/MR_20020205_EGD-0924/scans/3-T1_...,3-T1_3D,T1w,MR_20020205_EGD-0924,EGD-0924,False,/trinity/home/r098375/DDS/data/egd/NIFTI/MR_20...,/trinity/home/r098375/DDS/data/egd/NIFTI_SLICE...,0,0,/trinity/home/r098375/DDS/data/egd/NIFTI/MR_20...,4
4,data/egd/DICOM_STRUCTURED/MR_20020205/20020205...,1.2.826.0.1.3680043.10.104.2857427232523006061...,data/egd/data/MR_20020205_EGD-0924/scans/3-T1_...,3-T1_3D,T1w,MR_20020205_EGD-0924,EGD-0924,False,/trinity/home/r098375/DDS/data/egd/NIFTI/MR_20...,/trinity/home/r098375/DDS/data/egd/NIFTI_SLICE...,0,0,/trinity/home/r098375/DDS/data/egd/NIFTI/MR_20...,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47945,data/egd/DICOM_STRUCTURED/MR_97898/20180127/MR...,1.2.826.0.1.3680043.10.104.1081727604819600522...,data/egd/data/MR_20151117_EGD-1208/scans/450-A...,450-ADC__10__6_mm__s__Jun_26_2019_14_35_34,adc,MR_20151117_EGD-1208,EGD-1208,False,/trinity/home/r098375/DDS/data/egd/NIFTI/MR_97...,/trinity/home/r098375/DDS/data/egd/NIFTI_SLICE...,0,0,/trinity/home/r098375/DDS/data/egd/NIFTI/MR_97...,5
47946,data/egd/DICOM_STRUCTURED/MR_97898/20180127/MR...,1.2.826.0.1.3680043.10.104.1081727604819600522...,data/egd/data/MR_20151117_EGD-1208/scans/450-A...,450-ADC__10__6_mm__s__Jun_26_2019_14_35_34,adc,MR_20151117_EGD-1208,EGD-1208,False,/trinity/home/r098375/DDS/data/egd/NIFTI/MR_97...,/trinity/home/r098375/DDS/data/egd/NIFTI_SLICE...,0,0,/trinity/home/r098375/DDS/data/egd/NIFTI/MR_97...,18
47947,data/egd/DICOM_STRUCTURED/MR_97898/20180127/MR...,1.2.826.0.1.3680043.10.104.1081727604819600522...,data/egd/data/MR_20151117_EGD-1208/scans/450-A...,450-ADC__10__6_mm__s__Jun_26_2019_14_35_34,adc,MR_20151117_EGD-1208,EGD-1208,False,/trinity/home/r098375/DDS/data/egd/NIFTI/MR_97...,/trinity/home/r098375/DDS/data/egd/NIFTI_SLICE...,0,0,/trinity/home/r098375/DDS/data/egd/NIFTI/MR_97...,22
47948,data/egd/DICOM_STRUCTURED/MR_97898/20180127/MR...,1.2.826.0.1.3680043.10.104.1081727604819600522...,data/egd/data/MR_20151117_EGD-1208/scans/450-A...,450-ADC__10__6_mm__s__Jun_26_2019_14_35_34,adc,MR_20151117_EGD-1208,EGD-1208,False,/trinity/home/r098375/DDS/data/egd/NIFTI/MR_97...,/trinity/home/r098375/DDS/data/egd/NIFTI_SLICE...,0,0,/trinity/home/r098375/DDS/data/egd/NIFTI/MR_97...,14


In [37]:
df_new.describe()

,label,extra,slicenum
count,47950.0,47950.0,47950.000000
mean,0.0,0.0,12.000000
std,0.0,0.0,7.211178
min,0.0,0.0,0.000000
25%,0.0,0.0,6.000000
50%,0.0,0.0,12.000000
75%,0.0,0.0,18.000000
max,0.0,0.0,24.000000


In [38]:
df_new

,structured_path,filename,path,raw_name,name,label_exp,subject,train,NIFTI_path,ID,label,extra,NiftiPath,slicenum
0,data/egd/DICOM_STRUCTURED/MR_20020205/20020205...,1.2.826.0.1.3680043.10.104.2857427232523006061...,data/egd/data/MR_20020205_EGD-0924/scans/3-T1_...,3-T1_3D,T1w,MR_20020205_EGD-0924,EGD-0924,False,/trinity/home/r098375/DDS/data/egd/NIFTI/MR_20...,/trinity/home/r098375/DDS/data/egd/NIFTI_SLICE...,0,0,/trinity/home/r098375/DDS/data/egd/NIFTI/MR_20...,11
1,data/egd/DICOM_STRUCTURED/MR_20020205/20020205...,1.2.826.0.1.3680043.10.104.2857427232523006061...,data/egd/data/MR_20020205_EGD-0924/scans/3-T1_...,3-T1_3D,T1w,MR_20020205_EGD-0924,EGD-0924,False,/trinity/home/r098375/DDS/data/egd/NIFTI/MR_20...,/trinity/home/r098375/DDS/data/egd/NIFTI_SLICE...,0,0,/trinity/home/r098375/DDS/data/egd/NIFTI/MR_20...,8
2,data/egd/DICOM_STRUCTURED/MR_20020205/20020205...,1.2.826.0.1.3680043.10.104.2857427232523006061...,data/egd/data/MR_20020205_EGD-0924/scans/3-T1_...,3-T1_3D,T1w,MR_20020205_EGD-0924,EGD-0924,False,/trinity/home/r098375/DDS/data/egd/NIFTI/MR_20...,/trinity/home/r098375/DDS/data/egd/NIFTI_SLICE...,0,0,/trinity/home/r098375/DDS/data/egd/NIFTI/MR_20...,21
3,data/egd/DICOM_STRUCTURED/MR_20020205/20020205...,1.2.826.0.1.3680043.10.104.2857427232523006061...,data/egd/data/MR_20020205_EGD-0924/scans/3-T1_...,3-T1_3D,T1w,MR_20020205_EGD-0924,EGD-0924,False,/trinity/home/r098375/DDS/data/egd/NIFTI/MR_20...,/trinity/home/r098375/DDS/data/egd/NIFTI_SLICE...,0,0,/trinity/home/r098375/DDS/data/egd/NIFTI/MR_20...,4
4,data/egd/DICOM_STRUCTURED/MR_20020205/20020205...,1.2.826.0.1.3680043.10.104.2857427232523006061...,data/egd/data/MR_20020205_EGD-0924/scans/3-T1_...,3-T1_3D,T1w,MR_20020205_EGD-0924,EGD-0924,False,/trinity/home/r098375/DDS/data/egd/NIFTI/MR_20...,/trinity/home/r098375/DDS/data/egd/NIFTI_SLICE...,0,0,/trinity/home/r098375/DDS/data/egd/NIFTI/MR_20...,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47945,data/egd/DICOM_STRUCTURED/MR_97898/20180127/MR...,1.2.826.0.1.3680043.10.104.1081727604819600522...,data/egd/data/MR_20151117_EGD-1208/scans/450-A...,450-ADC__10__6_mm__s__Jun_26_2019_14_35_34,adc,MR_20151117_EGD-1208,EGD-1208,False,/trinity/home/r098375/DDS/data/egd/NIFTI/MR_97...,/trinity/home/r098375/DDS/data/egd/NIFTI_SLICE...,0,0,/trinity/home/r098375/DDS/data/egd/NIFTI/MR_97...,5
47946,data/egd/DICOM_STRUCTURED/MR_97898/20180127/MR...,1.2.826.0.1.3680043.10.104.1081727604819600522...,data/egd/data/MR_20151117_EGD-1208/scans/450-A...,450-ADC__10__6_mm__s__Jun_26_2019_14_35_34,adc,MR_20151117_EGD-1208,EGD-1208,False,/trinity/home/r098375/DDS/data/egd/NIFTI/MR_97...,/trinity/home/r098375/DDS/data/egd/NIFTI_SLICE...,0,0,/trinity/home/r098375/DDS/data/egd/NIFTI/MR_97...,18
47947,data/egd/DICOM_STRUCTURED/MR_97898/20180127/MR...,1.2.826.0.1.3680043.10.104.1081727604819600522...,data/egd/data/MR_20151117_EGD-1208/scans/450-A...,450-ADC__10__6_mm__s__Jun_26_2019_14_35_34,adc,MR_20151117_EGD-1208,EGD-1208,False,/trinity/home/r098375/DDS/data/egd/NIFTI/MR_97...,/trinity/home/r098375/DDS/data/egd/NIFTI_SLICE...,0,0,/trinity/home/r098375/DDS/data/egd/NIFTI/MR_97...,22
47948,data/egd/DICOM_STRUCTURED/MR_97898/20180127/MR...,1.2.826.0.1.3680043.10.104.1081727604819600522...,data/egd/data/MR_20151117_EGD-1208/scans/450-A...,450-ADC__10__6_mm__s__Jun_26_2019_14_35_34,adc,MR_20151117_EGD-1208,EGD-1208,False,/trinity/home/r098375/DDS/data/egd/NIFTI/MR_97...,/trinity/home/r098375/DDS/data/egd/NIFTI_SLICE...,0,0,/trinity/home/r098375/DDS/data/egd/NIFTI/MR_97...,14


In [39]:
labeldict = {"T1":0,"T1_c":1,"T2":2,"T2-FLAIR":3,"PD":4,
             "SWI":5, "GRE":6, "T2*":7, "DWI":8, "ADC":9, "BOLD":10,
             "FlowSensitve":11,"PWI":12,"ASL":13,
             "DTI":14,"Other":15}
def assignLabel(dataframe):
    description = dataframe['description']
    try:
        weight = dataframe['weighting']
    except:
        weight = None
    dropLabels = ['moco','mpr','hippo','spinal','unknown']
    otherCategories = ['fieldmap','scout','calibration','phase','localizer','nan','NotMri','pet','CT','e2','minIP','moco','other','survey', 'qflow','smartbrain'] #moco is a motion corrected series
    if description in dropLabels:
        return -1
    if description in otherCategories:
        return 15
    if description =='T1w':
        return 0
    if description =='T1c':
        return 1
    if weight:
        if description =='T2w':
            if weight=='T2':
                return 2
            if weight=='PD':
                return 4
    if description == 'T2w':
        return 2
    if 'FLAIR' in description:
        return 3
    if description =='pd':
        return 4
    if description =='swi':
        return 5
    if description.lower() =='gre':
        return 6
    if description =='T2star':
        return 7
    if description =='dwi':
        return 8
    if description =='adc':
        return 9
    if description =='bold':
        return 10
    if description =='angio': #flowsensitive
        return 11
    if description =='pwi' or description=='cbf':
        return 12
    if 'asl' in description:
        return 13
    if description == 'dti':
        return 14
    return -1

In [40]:
df_new['description'] = df_new['name']

In [41]:
df_new['label'] = df_new.apply(assignLabel, axis=1)

In [42]:
df_new['label'].value_counts()

label
 8     11325
 1      9625
 0      6825
-1      6775
 2      5775
 12     3150
 9      2825
 15     1500
 4        75
 13       50
 14       25
Name: count, dtype: int64

In [43]:
df_new[df_new['label']==-1]

,structured_path,filename,path,raw_name,name,label_exp,subject,train,NIFTI_path,ID,label,extra,NiftiPath,slicenum,description
125,data/egd/DICOM_STRUCTURED/MR_61373/20170605/MR...,1.2.826.0.1.3680043.10.104.1044043140781656433...,data/egd/data/MR_20150326_EGD-1215/scans/6-Sag...,6-Sag_Cube_FLAIR_FS_HS1_5,flair,MR_20150326_EGD-1215,EGD-1215,True,/trinity/home/r098375/DDS/data/egd/NIFTI/MR_61...,/trinity/home/r098375/DDS/data/egd/NIFTI_SLICE...,-1,0,/trinity/home/r098375/DDS/data/egd/NIFTI/MR_61...,3,flair
126,data/egd/DICOM_STRUCTURED/MR_61373/20170605/MR...,1.2.826.0.1.3680043.10.104.1044043140781656433...,data/egd/data/MR_20150326_EGD-1215/scans/6-Sag...,6-Sag_Cube_FLAIR_FS_HS1_5,flair,MR_20150326_EGD-1215,EGD-1215,True,/trinity/home/r098375/DDS/data/egd/NIFTI/MR_61...,/trinity/home/r098375/DDS/data/egd/NIFTI_SLICE...,-1,0,/trinity/home/r098375/DDS/data/egd/NIFTI/MR_61...,9,flair
127,data/egd/DICOM_STRUCTURED/MR_61373/20170605/MR...,1.2.826.0.1.3680043.10.104.1044043140781656433...,data/egd/data/MR_20150326_EGD-1215/scans/6-Sag...,6-Sag_Cube_FLAIR_FS_HS1_5,flair,MR_20150326_EGD-1215,EGD-1215,True,/trinity/home/r098375/DDS/data/egd/NIFTI/MR_61...,/trinity/home/r098375/DDS/data/egd/NIFTI_SLICE...,-1,0,/trinity/home/r098375/DDS/data/egd/NIFTI/MR_61...,10,flair
128,data/egd/DICOM_STRUCTURED/MR_61373/20170605/MR...,1.2.826.0.1.3680043.10.104.1044043140781656433...,data/egd/data/MR_20150326_EGD-1215/scans/6-Sag...,6-Sag_Cube_FLAIR_FS_HS1_5,flair,MR_20150326_EGD-1215,EGD-1215,True,/trinity/home/r098375/DDS/data/egd/NIFTI/MR_61...,/trinity/home/r098375/DDS/data/egd/NIFTI_SLICE...,-1,0,/trinity/home/r098375/DDS/data/egd/NIFTI/MR_61...,20,flair
129,data/egd/DICOM_STRUCTURED/MR_61373/20170605/MR...,1.2.826.0.1.3680043.10.104.1044043140781656433...,data/egd/data/MR_20150326_EGD-1215/scans/6-Sag...,6-Sag_Cube_FLAIR_FS_HS1_5,flair,MR_20150326_EGD-1215,EGD-1215,True,/trinity/home/r098375/DDS/data/egd/NIFTI/MR_61...,/trinity/home/r098375/DDS/data/egd/NIFTI_SLICE...,-1,0,/trinity/home/r098375/DDS/data/egd/NIFTI/MR_61...,11,flair
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47770,data/egd/DICOM_STRUCTURED/MR_97898/20180127/MR...,1.2.826.0.1.3680043.10.104.1081727604819600522...,data/egd/data/MR_20151117_EGD-1208/scans/6-Sag...,6-Sag_Cube_FLAIR_FS_HS1_5,flair,MR_20151117_EGD-1208,EGD-1208,False,/trinity/home/r098375/DDS/data/egd/NIFTI/MR_97...,/trinity/home/r098375/DDS/data/egd/NIFTI_SLICE...,-1,0,/trinity/home/r098375/DDS/data/egd/NIFTI/MR_97...,11,flair
47771,data/egd/DICOM_STRUCTURED/MR_97898/20180127/MR...,1.2.826.0.1.3680043.10.104.1081727604819600522...,data/egd/data/MR_20151117_EGD-1208/scans/6-Sag...,6-Sag_Cube_FLAIR_FS_HS1_5,flair,MR_20151117_EGD-1208,EGD-1208,False,/trinity/home/r098375/DDS/data/egd/NIFTI/MR_97...,/trinity/home/r098375/DDS/data/egd/NIFTI_SLICE...,-1,0,/trinity/home/r098375/DDS/data/egd/NIFTI/MR_97...,0,flair
47772,data/egd/DICOM_STRUCTURED/MR_97898/20180127/MR...,1.2.826.0.1.3680043.10.104.1081727604819600522...,data/egd/data/MR_20151117_EGD-1208/scans/6-Sag...,6-Sag_Cube_FLAIR_FS_HS1_5,flair,MR_20151117_EGD-1208,EGD-1208,False,/trinity/home/r098375/DDS/data/egd/NIFTI/MR_97...,/trinity/home/r098375/DDS/data/egd/NIFTI_SLICE...,-1,0,/trinity/home/r098375/DDS/data/egd/NIFTI/MR_97...,10,flair
47773,data/egd/DICOM_STRUCTURED/MR_97898/20180127/MR...,1.2.826.0.1.3680043.10.104.1081727604819600522...,data/egd/data/MR_20151117_EGD-1208/scans/6-Sag...,6-Sag_Cube_FLAIR_FS_HS1_5,flair,MR_20151117_EGD-1208,EGD-1208,False,/trinity/home/r098375/DDS/data/egd/NIFTI/MR_97...,/trinity/home/r098375/DDS/data/egd/NIFTI_SLICE...,-1,0,/trinity/home/r098375/DDS/data/egd/NIFTI/MR_97...,17,flair


In [44]:
df_new = df_new[df_new['label']!=-1]

In [45]:
testLabelFrame = df_new[['ID','label','extra']][df_new['train']!=True]
trainLabelFrame = df_new[['ID','label','extra']][df_new['train']]
AllLabelFrame = df_new[['ID','label','extra']]

In [46]:
trainLabelFrame['label'].value_counts()

label
8     10050
1      8625
0      6075
2      5250
12     2625
9      2325
15     1250
4        75
14       25
13       25
Name: count, dtype: int64

In [47]:
testLabelFrame['label'].value_counts()

label
8     1275
1     1000
0      750
2      525
12     525
9      500
15     250
13      25
Name: count, dtype: int64

In [48]:
trainLabelFrame.to_csv('egd_trainLabels_extensive3.txt', header=False,index=False, sep='\t')
testLabelFrame.to_csv('egd_testLabels_extensive3.txt', header=False,index=False, sep='\t')
AllLabelFrame.to_csv('egd_Labels_extensive3.txt', header=False,index=False, sep='\t')